In [6]:
from typing import Optional, List
# simulate a data with 10 columns , timestamp, worker id {1,10} , Travel time, Assignment duration , day of the week, month ,location , My plan is to train a ML model on this
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

class Generator():
  def __init__(self, n_operators: int = 5, location_list: Optional[List[str]] = None, ):
    self.n_operators = n_operators
    self.location_list = location_list
    if self.location_list is None:
      self.location_list = [f"location_{i}" for i in "abcdef"]
    pass

  def generate_data(self, num_rows: int=100000, months: int = 1):
      start_date = datetime(2023, 1, 1)
      # end_date = datetime(2023, 1+ months, 31)

      data = []
      for _ in range(num_rows):
          hour_of_day = random.randint(8, 17)
          timestamp = start_date + timedelta(days=random.randint(0, 6),
                                            hours=hour_of_day,
                                            minutes=random.randint(0, 59))
          
          worker_id = random.randint(1, self.n_operators)
          
          travel_time = random.randint(15, 120)  # in seconds

          assignment_duration = travel_time + random.randint(300, 600)  # in seconds

          # Special Cases
          if(worker_id in [1]):
            travel_time = 90 + random.randint(-10,10)
            assignment_duration = 500 + random.randint(-40,40)
          elif(hour_of_day in [15,16,17]):
            travel_time = 100 + random.randint(-10,10)
            assignment_duration = 450 + random.randint(-40,40)
          else:
            pass
          
          day_of_week = timestamp.weekday()
          month = timestamp.month
          location = random.choice(self.location_list)

          data.append([timestamp, worker_id, travel_time, assignment_duration,
                      day_of_week, hour_of_day, month, location])
      
      # randomly append 20 anomaly data points and shuffle
      for _ in range(3):
        hour_of_day = random.randint(8, 17)
        travel_time = random.randint(15, 120)
        assignment_duration = travel_time + random.randint(300, 600)

        anomaly_choice = random.randint(1, 3)
        if anomaly_choice == 1:
          hour_of_day = random.randint(19, 21)
        elif anomaly_choice >= 2:
          travel_time = random.randint(15000, 17000)
        else:
          assignment_duration = random.randint(50, 60)

        timestamp = start_date + timedelta(days=random.randint(0, 6),
                                            hours=hour_of_day,
                                            minutes=random.randint(0, 59))
        worker_id = random.randint(1, self.n_operators)
        day_of_week = timestamp.weekday()
        month = timestamp.month
        location = random.choice(self.location_list)
        data.append([timestamp, worker_id, travel_time, assignment_duration, day_of_week, hour_of_day, month, location])  # Anomaly: travel_time=900, assignment_duration=1000
      
      random.shuffle(data)

      df = pd.DataFrame(data,
                        columns=["timestamp", "worker_id", "travel_time",
                                "assignment_duration", "day_of_week", "hour_of_day" , "month",
                                "location"])
      return df

gnr = Generator(4)

df = gnr.generate_data()
display(df.head())


ModuleNotFoundError: No module named 'pandas'

In [ ]:
from sklearn.ensemble import IsolationForest

# Prepare the data for the model
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['timestamp'].dt.hour

import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming 'df' is your DataFrame

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the selected columns
scaler.fit(df[['travel_time', 'assignment_duration']])

# Transform the selected columns using the scaler
df[['travel_time_scaled', 'assignment_duration_scaled']] = scaler.transform(df[['travel_time', 'assignment_duration']])

# Drop the original columns if desired
# df = df.drop(columns=['travel_time', 'assignment_duration'])

# Display the updated DataFrame
display(df.head())
# df = pd.get_dummies(df, columns=['location'])

# Assuming 'df' is your DataFrame
# categorical_features = ['hour', 'day_of_week', 'worker_id'] 

# # Create a OneHotEncoder instance
# encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # sparse=False for dense output
# # encoder = LabelEncoder()  # sparse=False for dense output

# # Fit the encoder to the categorical features
# encoder.fit(df[categorical_features])

# # Transform the categorical features into one-hot encoded features
# encoded_features = encoder.transform(df[categorical_features]).astype(int)

# # Create column names for the encoded features
# encoded_feature_names = encoder.get_feature_names_out(categorical_features)

# # Create a DataFrame for the encoded features
# encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names, index=df.index)

# # Concatenate the encoded features with the original DataFrame
# df = pd.concat([df, encoded_df], axis=1)

# # Drop the original categorical features
# df = df.drop(columns=categorical_features)
# df = df.drop(columns = ['timestamp', 'hour_of_day'])
# X = df[:]

# Display the preprocessed DataFrame

### Encode locations
location_encoder = LabelEncoder()

# Fit the encoder to the 'location' column
location_encoder.fit(df['location'])

# Transform the 'location' column using the encoder
df['location_encoded'] = location_encoder.transform(df['location'])

# Drop the original 'location' column
df = df.drop(columns=['location'])



features = ['worker_id', 'travel_time_scaled', 'assignment_duration_scaled', 'day_of_week', 'month', 'hour'] + [col for col in df.columns if 'location' in col]
X = df[features]
display(X.head())



# print(X)


In [ ]:
# Train the Isolation Forest model
print(X.columns)
model = IsolationForest(contamination='auto') # Adjust contamination as needed
model.fit(X)

# Get anomaly predictions
df['anomaly_score'] = model.decision_function(X)
df['anomaly'] = model.predict(X)

# Display the results
display(df.head())

In [ ]:
# Set a threshold for anomaly score
threshold = -0.005  # Adjust this value as needed

# Filter data points with anomaly scores below the threshold
high_anomalies = df[df['anomaly_score'] < threshold]

# Display the high anomalies
# display(high_anomalies)

#sort by anomalies
df_sorted = df.sort_values(by=['anomaly_score'])
top_10_anomalies = df_sorted.head(20)
bottom_10_anomalies = df_sorted.tail(20)
# display(df.tail(20))
display(top_10_anomalies)

display(bottom_10_anomalies)

